<center>
    <h1 style="font-size: 12; font-weight: 900; ">--- 4 ---<h1>
    <h2>CSiAPI - ETABS y SAP2000</h2>
    <!-- <p><strong></strong</p> -->
</center>

In [ ]:
import os, sys
import comtypes.client

In [ ]:
# Variable para definir una instancia manual para Etabs.
rutaEspecifica = False

# si la bandera anterior se establece en True, entonces debe especificar la ruta a ETABS a continuación
ProgramPath = "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe"

try:
    ETABSObject = comtypes.client.GetActiveObject("CSI.ETABS.API.ETABSObject")
    print("Coneccion exitosa!.\nadjuntando a una instancia existente.")
except (OSError, comtypes.COMError):
    print("No se encontró ninguna instancia en ejecución del programa(Etabs).")

    print("Tratando de Ejecutar etabs!.")
    #create API helper object
    helper = comtypes.client.CreateObject('ETABSv1.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
    # helper = comtypes.client.CreateObject('ETABSv1.Helper').QueryInterface(comtypes.gen.ETABSv1.cHelper)
    if rutaEspecifica: # para versiones anteriores a la ultima version instalada
        try:
            ETABSObject = helper.CreateObject(ProgramPath)
            print("Coneccion esitosa!.\nConexion Manual")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            ETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
            print("Coneccion esitosa!.")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    print("Ejecutando etabs!.  ultima version instalada")
    #start ETABS application | ejecutar la Aplicacion ETABS.
    ETABSObject.ApplicationStart()


smodel = ETABSObject.SapModel

In [ ]:
kgf_cm_C=14
# string Name
# eMatType MatType
# int Color = -1
# string Notes = ""
# string GUID = ""
# smodel.SetPresentUnits(kgf_cm_C)

smodel.InitializeNewModel(kgf_cm_C)

# int NumberStorys | numero de pisos
# double TypicalStoryHeight | altura tipico de piso
# double BottomStoryHeight  | altura de piso de sotano/abajo
# int NumberLinesX  | numero de lineas en x, en el plano
# int NumberLinesY  | numero de lineas en y, en el plano
# double SpacingX   | espaciado en x, en plano
# double SpacingY   | espaciado en y, en plano
nmodel = smodel.File.NewGridOnly(4,10,8,4,4,10,10)

#
name_material="fc=280 kgf/cm2"
smodel.PropMaterial.SetMaterial(name_material, 2, -1)
#
smodel.PropMaterial.SetOConcrete_1(name_material, 280, False, 0, 1, 2, 0.0022, 0.0052, -0.1, 0, 0)
name_materail_rebar = "A615Gr60"
smodel.PropMaterial.SetMaterial(name_materail_rebar, 6)

kgf_m_C=8
smodel.SetPresentUnits(kgf_cm_C)

# Sections | secciones
sectionName = 'Viga 30x60'
smodel.PropFrame.SetRectangle(sectionName, name_material, 0.65, 0.30)

# falta una configuracion aquii

# Patterns | patrones
ret = smodel.LoadPatterns.Add("SCM", 2)
ret = smodel.LoadPatterns.Add("Dead", 1)
ret = smodel.LoadPatterns.Add("Live", 3)
 
# Cases | casos
ret = smodel.LoadCases.StaticLinear.SetCase("SCM")
ret = smodel.LoadCases.StaticLinear.SetCase("Dead")
ret = smodel.LoadCases.StaticLinear.SetCase("Live")

# Combinations | combinaciones
ret = smodel.RespCombo.Add("UDCon1", 0)
ret = smodel.RespCombo.Add("UDCon2", 0)

CNameType = 0
ret = smodel.RespCombo.SetCaseList("UDCon1", CNameType, "Dead", 1.4)
ret = smodel.RespCombo.SetCaseList("UDCon1", CNameType, "SCM", 1.4)

ret = smodel.RespCombo.SetCaseList("UDCon2", CNameType, "Dead", 1.2)
ret = smodel.RespCombo.SetCaseList("UDCon2", CNameType, "Live", 1.6)
ret = smodel.RespCombo.SetCaseList("UDCon2", CNameType, "SCM", 1.2)

# NumberItems = 0
# CNameType = []
# CName = []
# sfSF = []
# smodel.RespCombo.GetCaseList("UDCon1", NumberItems, CNameType, CName, sfSF)

# Elements | elementos
FrameName1 = ' '
[FrameName1, ret] = smodel.FrameObj.AddByCoord(304.8, 0, 243.84, 609.6, 0, 243.84, FrameName1, sectionName, '1', 'Global')

# Assigning restraints
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, False, False, False] #UX,UY,UZ, MX,MY,MZ
[PointName1, PointName2, ret] = smodel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = smodel.PointObj.SetRestraint(PointName1, Restraint)

Restraint = [False, False, True, False, False, False] #UX,UY,UZ, MX,MY,MZ
[PointName1, PointName2, ret] = smodel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = smodel.PointObj.SetRestraint(PointName2, Restraint)

smodel.View.RefreshView();

# cambio de orientacion.
# vName="v1"
# vCardinalPoint=2
# vMirror2=False
# vStiffTransform=False
# vOffset1=[]
# vOffset2=[]
# ret = smodel.FrameObj.SetInsertionPoint(vName, vCardinalPoint, vMirror2, vStiffTransform, vOffset1, vOffset2)

# Se definen registros de salida
projectPath = r'C:\\CSi_API_Example';
if not os.path.exists(projectPath):
    try: os.makedirs(projectPath)
    except OSError: pass
ModelPath = projectPath + os.sep + 'Beam3D.edb'
# save model | guardar nuestro modelo
ret = smodel.File.Save(ModelPath)

# run model (this will create the analysis model)
ret = smodel.Analyze.RunAnalysis()

## Referencias.

- aa